# Prompt engineering



## wat

Prompt engineering is het sleutelen aan de vraag (de prompt) die je aan LLM's stelt zodat je een beter antwoord krijgt.

Deze notebook toont een aantal voorbeelden m.b.t. prompt engineering.
Het gaat hem erom enkele prompt engineering technieken te demonstreren a.d.h.v. deze voorbeelden.

De code dateert van 2024 en dient hoogst waarschijnlijk aangepast te worden om te functioneren. 
Voor het OpenAI gedeelte is er ook een (betalende) API-key nodig.

Om de OpenAI API key te gebruiken, kan je starten met deze documentatie: \
https://platform.openai.com/docs/guides/text?api-mode=responses

In [ ]:
# Met onderstaande code creëer je een openAI client en een chat completion.
import os
from openai import OpenAI

# Indien je met colab werkt, gebruik dan:
# from google.colab import userdata
# client = OpenAI(
#     api_key=userdata.get('OPENAI_API_KEY'),
# )

# Werk je met een .env file, gebruik dan:
from dotenv import load_dotenv
import os
_ = load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")  

assert isinstance(api_key, str) and api_key.startswith("sk-"), "API key not a string"

client = OpenAI(api_key=api_key.strip())


# Je kan hiermee testen of het werkt:
"""
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-3.5-turbo",
)
"""

sk-sv


'\nchat_completion = client.chat.completions.create(\n    messages=[\n        {\n            "role": "user",\n            "content": "Say this is a test",\n        }\n    ],\n    model="gpt-3.5-turbo",\n)\n'

### OpenAI API

Om de API te gebruiken dien je de [chat completions API](https://platform.openai.com/docs/guides/gpt/chat-completions-api) aan te spreken. Je geeft minimaal een (lijst van) boodschappen, het model dat je wenst te gebruiken en de api key als input, en krijgt het door het model gegenereerde antwoord terug.

Hoewel *Chat*GPT eigenlijk ontworpen is om conversaties te houden, kan het toch nuttig zijn als je het in een "single turn" mode gebruikt ook.

(In deze notebook gebruiken we altijd `role: user` gebruiken, verder gaan we dieper in op welke rollen er zijn en hoe die kunnen gebruikt worden)

Er zijn verschillende openai modellen (ref. [openai models](https://platform.openai.com/docs/models)). Hier maken we gebruik van het model "gpt-3.5-turbo".

In [ ]:
response = openai.ChatCompletion.create(model="gpt-3.5-turbo", 
                                        messages=[{"role": "user", 
                                                   "content": "Ik zou graag meer leren over prompt engineering"}])
print (response)

{
  "id": "chatcmpl-ACpOHMJ6iUT8o7Ic7wqifCX9PhrNO",
  "object": "chat.completion",
  "created": 1727620845,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Prompt engineering is een concept dat voortkomt uit de softwareontwikkeling en is gericht op het cre\u00ebren van een effici\u00ebnte en gestructureerde ontwikkelomgeving. Het is een strategie waarbij ontwikkelaars worden aangemoedigd om snel te reageren op feedback en verzoeken van gebruikers. Prompt engineering omvat het analyseren, ontwerpen, implementeren en testen van software met een snelle doorlooptijd.\n\nEen belangrijk aspect van prompt engineering is het gebruik van agile ontwikkelmethoden, zoals Scrum en Kanban, waarbij teams in korte iteraties werken om regelmatig updates en verbeteringen uit te brengen. Door deze aanpak kunnen ontwikkelaars snel inspelen op veranderingen en feedback van gebruikers, waardoor de software sneller en

Je krijgt steeds een JSON object als antwoord, naast een timestamp en de precieze versie van het model dat gebruikt werd interesseert ons hier vooral het object `choices[0].message["content"]` dat het antwoord bevat.

Daarnaast zal ook altijd het aantal tokens voor vraag/antwoord terug gestuurd worden, waarmee je ook de kost kan berekenen. Deze specifieke prompt kostte (in 2023) ~ €0,0012835.

In de cursus van vorig jaar werd hier verwezen naar:
ref. https://platform.openai.com/docs/guides/chat-completions, secties "Overview", "Getting Started" en "Response format".

De link werkt nog steeds, maar de secties zijn veranderd, alsook het advies over hoe te prompten + de vorm van de output van de modellen.

We schrijven een helper functie "get_answer". Hiermee wordt het mogelijk om snel te wisselen tussen het gebruik van een openai model of een Huggingface model. In de helper functie staat de parameter "max_tokens" standaard op 100.

In [3]:
# model options are for example: "gpt-3.5-turbo", "gpt-4.o-mini"
def get_answer(prompt, model="gpt-3.5-turbo", role="user", max_tokens=100):
    message = [{"role": role, "content": prompt}]
    res = openai.ChatCompletion.create(
    model=model,
    messages=message,
    max_tokens=max_tokens
        )
    return res.choices[0].message.content

# model options are for example: 
#   "meta-llama/Meta-Llama-3-8B-Instruct"
#   "NousResearch/Hermes-3-Llama-3.1-8B"

from huggingface_hub import InferenceClient
from dotenv import load_dotenv
import os

_ = load_dotenv()
hf_api_key = os.getenv('HF_API_KEY')

assert isinstance(hf_api_key, str) and len(hf_api_key) > 10, "HF API key not a string"

def get_answer_hf(prompt, model="meta-llama/Meta-Llama-3-8B-Instruct", role="user", max_tokens=100, hf_api_key=hf_api_key):
    client = InferenceClient(model, api_key=hf_api_key)
    message = [{"role": role, "content": prompt}]
    res = client.chat_completion(messages=message, max_tokens=max_tokens)
    return res.choices[0].message.content, res.choices[0].message

#### Oefening
Beschouw onderstaande alternatieve helper functie. I.p.v. enkel het textueel resultaat terug te geven, geven we een ruimer overzicht.

In [8]:
# model options are for example: "gpt-3.5-turbo", "gpt-4.o-mini"
def get_answer_alternative(prompt, model="gpt-3.5-turbo", role="user", max_tokens=3):
    message = [{"role": role, "content": prompt}]
    res = openai.ChatCompletion.create(
    model=model,
    messages=message,
    max_tokens=max_tokens
        )
    return res.choices[0].message.content, res.choices[0]

In [9]:
# test get_answer_alternative("What is the capital of France?")
text_completion, entire_completion = get_answer_alternative("What is the capital of France?")
print(f"text_completion: {text_completion}")
print(f"text_completion: {entire_completion}")

text_completion: The capital of
text_completion: {
  "index": 0,
  "message": {
    "role": "assistant",
    "content": "The capital of",
    "refusal": null
  },
  "logprobs": null,
  "finish_reason": "length"
}


- Welke finish_reason krijg je wanneer je de optie max_tokens = 10, en wanneer je de optie max_tokens = 3 gebruikt? 
- Wat betekent dit?
- Wat zijn andere finish reasons?

In [39]:
# Test your openAI access
print(get_answer("What is the capital of France?"))
print(get_answer("What is the capital of France?", model="gpt-4o-mini"))

The capital of France is Paris.
The capital of France is Paris.


In [4]:
# Setup and test your HuggingFace access
from huggingface_hub import InferenceClient
print(get_answer_hf("What is the capital of France?"))
print(get_answer_hf("What is the capital of France?", model="NousResearch/Hermes-3-Llama-3.1-8B"))

('The capital of France is Paris.', ChatCompletionOutputMessage(role='assistant', content='The capital of France is Paris.', tool_call_id=None, tool_calls=None))
("The capital of France is Paris. Paris has been the capital of France since the 12th century and is the country's major cultural, commercial, and industrial center. It is also a global hub for fashion, art, and gastronomy. The city is known for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, and the Notre-Dame Cathedral. Paris is situated on the River Seine and is the most populous city in France, with an estimated population of over", ChatCompletionOutputMessage(role='assistant', content="The capital of France is Paris. Paris has been the capital of France since the 12th century and is the country's major cultural, commercial, and industrial center. It is also a global hub for fashion, art, and gastronomy. The city is known for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, and the Notre-

## prompt technieken - principes



De voorbeelden (en prompts) zullen soms in het Engels zijn. Hoewel Nederlands vaak redelijk werkt, zijn de resultaten toch veel betrouwbaarder en nuttiger in het Engels. Zowel bij ChatGPT 3.5, maar vooral wanneer we bv. een model lokaal draaien, of wanneer we een kleiner model gebruiken.
Ondertussen is er een duidelijke vooruitgang in de talenkennis met ChatGPT4o-mini (gebruik deze voor alledaagse taken) en ChatGPT4o (gebruik deze voor complexere taken).


### categorie 1: input
Input data is de data die beschrijft waar het over gaat bij de prompt, de tekst die moet samengevat worden, de bachelorproef die moet geëvalueerd worden, het stuk code waar uitleg over moet gegeven worden.
#### input - gebruik scheidingstekens
Gebruik scheidingstekens om bepaalde delen van de vraag duidelijk af te bakenen, scheidingstekens zijn bvb. ``` of """ of <><>


In [ ]:
text = f"""
Prompt engineering is tinkering with the question \
(the prompt) you ask LLMs so that you get a better answer. 

It soon became apparent that with ChatGPT etc., the usability \
and reliability of the answer can be greatly improved by asking \
the question in a certain way. A new discipline was thus born: \
prompt engineering. 

The interest in this is very new, and as the models mature more \
and more consensus of what "best practices" are will emerge here \
as well, but we can already establish some principles on how to \
get better results from these models, depending on what you are \
trying to achieve. 
The chances that this chapter will be the same \
next year (or even by the end of the semester) are pretty slim, \
this area of research is changing very quickly.
"""

prompt = f"""
Summarize the text delimited by triple backticks into a single sentence \
```{text}```
"""

res = get_answer(prompt)
print(res)

Prompt engineering is a new discipline that involves tailoring questions asked to language models like ChatGPT in order to improve the usability and reliability of the answers, and although the field is still evolving, there are already some principles established on how to achieve better results from these models.


### categorie 2: instructies 
De instructies beschrijven wat het LLM moet doen.
"Geef 5 aanbevelingen voor nieuwe boeken", "Beschrijf alle schrijf- en grammaticale fouten, geef twee verbeterpunten aan waar de student mee aan de slag kan", "Leg uit wat dit stuk code doet"
#### instructies: geef duidelijke, concrete instructies
Geef duidelijke, concrete instructies.

- Een voorbeeld van een vage instructie: \
prompt = f"""You will be presented with a news article. Extract useful information in a structured format. """

- Een voorbeeld van een duidelijke, concrete instructie: \
prompt = f"""You will be presented with a news article. Your task it to identify any opininions expressed about the government, and their sentiment.
"""

#### instructies: vraag gestructureerde output
LLM's hebben een goede notie van hoe correcte JSON (of zelfs HTML) er uit ziet, door een skelet van de gevraagde output te voorzien kan je veel beter gestructureerde output krijgen (die je dan weer veel vlotter kan verwerken met volgende prompts of in andere code)

In [85]:
prompt = f"""
Generate an array of three bank transactions, provide them in JSON format \
with the following keys: transaction_id, amount, date, place \ 
Only output the json object
"""

res = get_answer(prompt)
json_data = res
print(res)

[
  {
    "transaction_id": "1",
    "amount": 50.00,
    "date": "2022-01-10",
    "place": "Grocery Store"
  },
  {
    "transaction_id": "2",
    "amount": 25.50,
    "date": "2022-01-12",
    "place": "Gas Station"
  },
  {
    "transaction_id": "3",
    "amount": 100.75,
    "date": "2022-01-15",
    "place": "Restaurant"
  }
]


#### instructies: split complexe taken op in eenvoudigere taken

In [120]:
# article = "'It is not for Rishi Sunak to decide that negociations are over before he has even stepped in the room', they added."
article = "This dispute will end only at the negotiating table. \
If the PM was hoping to demoralise and divide our profession with his actions, he will be disappointed."

In [121]:
prompt = f"""You are presented with a news article: {article}. \
Your task is to identify any opinions expressed about the government, and their sentiment. \
Approach this task step-by-step, take your time and do not skip steps. \
The steps to follow are the following:
1. Read a paragraph of the news article
2. Determine whether an opinion is expressed in this paragraph. If not, continue to the next paragraph.
3. If there is an opinion, extract a JSON with keys as follows:
- opinion: allowable values are "positive", "negative" or "neutral"
- evidence: contains a list of strings evidencing the opinion
- speaker: the person or government body who expressed the opinion.
"""

In [122]:
res = get_answer(prompt)
print(res)

1. "This dispute will end only at the negotiating table. If the PM was hoping to demoralise and divide our profession with his actions, he will be disappointed."

2. Yes, an opinion is expressed in this paragraph.

3. {
    "opinion": "negative",
    "evidence": [
        "If the PM was hoping to demoralise and divide our profession with his actions, he will be disappointed."
    ],
    "speaker": "unknown"
}


#### instructies: laat het model eerst zelf een oplossing uitwerken, alvorens een conclusie te trekken

In [87]:
prompt = f"""
Bepaal of het antwoord van de student correct is of niet.

Vraag:
Er wordt een opslagplaats gebouwd, de bouwkost wordt geraamd op €150 \
per vierkante meter.
De grond zelf kost €200 per vierkante meter
Het contract om het gebouw te bewaken is eenmalig €5000 en \
daarbovenop €10 per vierkante meter per jaar.

Hoeveel kost het gebouw mij het eerste jaar?

Oplossing van de student:
Stel dat x het aantal vierkante meter is
Kost van de grond: 200x
Bouwkost: 150x
Bewakingsfirma: 5000 + 100x
Totale kost: 200x + 150x + 5000 + 100x = 450x + 5000
"""
response = get_answer(prompt)
print(response)

Het antwoord van de student is correct.


Merk op dat het antwoord van de student helemaal niet correct is, er is een (typ?)fout in de redenering, de bewakingsfirma kost 5000+10x, niet 5000+100x

We kunnen hier betere resultaten bekomen door het model eerst zelf een volledige oplossing te laten uitwerken en dan het resultaat te vergelijken.

In [88]:
prompt = f"""
Bepaal of het antwoord van de student correct is of niet.

Om dit te bepalen moet je de volgende stappen ondernemen:
- Werk eerst zelf de volledige oplossing uit
- Vergelijk vervolgens jouw antwoord met dat van de student om \
te kijken of de student zijn antwoord correct is of niet
Bepaal niet of het antwoord van de student correct is alvorens \ 
je de oplossing volledig uitgewerkt hebt.

Geef als uitvoer jouw uitgewerkte oplossing, gevolgd door \
het antwoord op de vraag: "is de oplossing van de student dezelfde als die van jou"

Vraag:
Er wordt een opslagplaats gebouwd, de bouwkost wordt geraamd op €150 \
per vierkante meter.
De grond zelf kost €200 per vierkante meter
Het contract om het gebouw te bewaken is eenmalig €5000 en \
daarbovenop €10 per vierkante meter per jaar.

Hoeveel kost het gebouw mij het eerste jaar?

Oplossing van de student:
Stel dat x het aantal vierkante meter is
Kost van de grond: 200x
Bouwkost: 150x
Bewakingsfirma: 5000 + 100x
Totale kost: 200x + 150x + 5000 + 100x = 450x + 5000
"""
response = get_answer(prompt)
print(response)

Uitwerking van de oplossing:

Kost van de grond: 200 euro per vierkante meter
Kost van de bouw: 150 euro per vierkante meter
Kost van de bewakingsfirma: eenmalig 5000 euro + 10 euro per vierkante meter

De totale kost voor het gebouw het eerste jaar is dus:
Totaal = (200 + 150 + 10) * x + 5000
Totaal = 360x + 5000

Dus, de totale kost voor het gebouw het eerste jaar bedraagt 360x + 5000 euro.

Het antwoord van de student is niet correct, aangezien de totale kost van het gebouw het eerste jaar 360x + 5000 euro is, terwijl de student 450x + 5000 euro heeft als totale kost. 

Antwoord: Nee


Nu is het antwoord wel juist.

#### instructies: geef het model tijd om te denken

Een voorbeeld van een prompt:

prompt = f"""Approach this task step-by-step, take your time and do not skip steps."""

### categorie 3: examples

Een krachtige strategie om het model betere resultaten te laten produceren, is door voorbeelden van de taak die je wilt dat het model uitvoert, op te nemen in de prompt.

Voorbeelden geven binnen het context window wordt 'in-context learning' genoemd. Met in-context learning kun je LLM's meer helpen leren over de gevraagde taak door voorbeelden of extra gegevens in de prompt op te nemen. 

Hier is een concreet voorbeeld.
Binnen de getoonde prompt vraag je het model om het sentiment van een recensie te classificeren. 
De prompt bestaat uit:
- een instructie: 'Classify this review',
- de inhoud: 'I loved this movie!',
- output format: 'Sentiment:'

In [19]:
# ChatGPT4o-mini
response = get_answer("Classify this review: I loved this movie! Sentiment:")
print(response)

Sentiment: Positive


Deze methode, inclusief je invoergegevens binnen de prompt, wordt **'zero-shot inference'** genoemd. De grootste LLM's zijn verrassend goed in dit opzicht, ze begrijpen de te voltooien taak en geven een goed antwoord. In dit voorbeeld identificeert het model het sentiment correct als positief. Kleinere modellen kunnen hier echter moeite mee hebben. Hieronder is een voorbeeld. Zoals je kunt zien, volgt het model de instructie niet. Hoewel het tekst genereert die enigszins gerelateerd is aan de prompt, kan het model de details van de taak niet achterhalen en identificeert het het sentiment niet. Dit is waar het geven van een voorbeeld binnen de prompt de prestaties kan verbeteren. 

![in_context_learning_zero_shot_inference](img/in_context_learning_zero_shot_inference.png)

In de figuur hieronder is de prompttekst langer. De prompttekst begint met een voorbeeld dat de taken die uitgevoerd moeten worden aan het model demonstreert. Ik vond deze film geweldig, gevolgd door een voltooide sentimentanalyse. In dit geval is de recensie positief.

Vervolgens geeft de prompt de instructie opnieuw en bevat de daadwerkelijke invoerrecensie die we willen dat het model analyseert. Je geeft deze nieuwe langere prompt door aan het kleinere model, dat nu een betere kans heeft om de taak die je specificeert en het formaat van de gewenste respons te begrijpen. De opname van één voorbeeld staat bekend als **'one-shot inference'**, in tegenstelling tot de zero-shot prompt die je eerder gaf.

![in_context_learning_one_shot_inference](img/in_context_learning_one_shot_inference.png)

Soms zal één voorbeeld niet voldoende zijn voor het model om te leren wat je wilt dat het doet. Je kan dan meerdere voorbeelden meegeven om het model nog beter te sturen. Het meegeven van meerdere voorbeelden heet **"few shot inference"**.

#### Samengevat:
Je kunt je prompts ontwerpen om het model aan te moedigen te leren door voorbeelden. Terwijl de grootste modellen goed zijn in zero-shot inference zonder voorbeelden, kunnen kleinere modellen profiteren van one-shot of few-shot inference die voorbeelden van het gewenste gedrag bevatten. 

Je bent hierbij gelimiteerd door de 'context window',  de hoeveelheid in-context learning die je in het model kunt doorgeven is beperkt. Over het algemeen, als je merkt dat je model niet goed presteert wanneer je bijvoorbeeld vijf of zes voorbeelden opneemt, moet je proberen je model te finetunen.

### categorie 4: constraints

De constraints dienen om af te lijnen waartoe het LLM zich moet beperken. Dit kan gaan om vormelijke beperkingen "Genereer enkel een JSON file met key 'boek' en key 'author'", "Maximaal 100 woorden lang", "in de stijl van Shakespeare".

Maar er kunnen ook inhoudelijke beperkingen zijn: "Wees vriendelijk", "Eindig met een positieve noot", "Zorg dat de boeken van hetzelfde genre zijn".

In [23]:
response = get_answer("Schrijf python code die een histogram maakt van volgende data: {jaar:[2022, 2023, 2024], frequency:[0.4, 0.6, 0.65]}")

In [24]:
print(response)

Natuurlijk! Hier is een voorbeeld van Python-code die een histogram genereert op basis van de gegeven data. We maken gebruik van de populaire bibliotheek `matplotlib` om het histogram te plotten.

Zorg ervoor dat je `matplotlib` hebt geïnstalleerd. Je kunt het installeren met pip als dat nog niet gedaan is:

```bash
pip install matplotlib
```

Hier is de code:

```python
import matplotlib.pyplot as plt

# Gegeven data
data = {
    'jaar': [2022, 2023, 2024],
    'frequency': [0.4, 0.6, 0.65]
}

# Plot het histogram
plt.bar(data['jaar'], data['frequency'], color='skyblue')

# Voeg titels en labels toe
plt.title('Histogram van Frequenties per Jaar')
plt.xlabel('Jaar')
plt.ylabel('Frequentie')

# Toon de grafiek
plt.xticks(data['jaar'])  # Zorg ervoor dat de jaren goed op de x-as staan
plt.ylim(0, 1)  # Zet de y-as limieten van 0 tot 1, omdat de frequenties tussen 0 en 1 liggen
plt.grid(axis='y')

# Laat de grafiek zien
plt.show()
```

Bij het uitvoeren van deze code krijg je een histogra

In [25]:
response = get_answer("Schrijf python code die een histogram maakt van volgende data: {jaar:[2022, 2023, 2024], \
frequency:[0.4, 0.6, 0.65]}. Beperk de output to het weergeven van de Python code.")

In [26]:
print(response)

```python
import matplotlib.pyplot as plt

data = {
    'jaar': [2022, 2023, 2024],
    'frequency': [0.4, 0.6, 0.65]
}

plt.bar(data['jaar'], data['frequency'], color='blue')
plt.xlabel('Jaar')
plt.ylabel('Frequentie')
plt.title('Histogram van Frequentie per Jaar')
plt.xticks(data['jaar'])
plt.ylim(0, 1)
plt.show()
```


### categorie 5: context or role

De context van een prompt beschrijft meestal op welke manier het LLM zich dient te gedragen, "Je bent een expert JavaScript developer", "Je bent een AI assistent die boeken aanraadt op basis van gebruikers voorkeuren", "Je bent een hogeschool lector die bachelorproefvoorstellen evalueert"

In [123]:
prompt = f"""
You're an expert JavaScript Developer. Construct a GET request handler which returns a json in the following format ```{json_data}``` when requesting the path /transactions using NodeJs.
only output the code without formatting
"""

response = get_answer(prompt)
code = response;
print(response)

const express = require('express');
const app = express();

app.get('/transactions', (req, res) => {
  const transactions = [
    {
      "transaction_id": "1",
      "amount": 50.00,
      "date": "2022-01-10",
      "place": "Grocery Store"
    },
    {
      "transaction_id": "2",
      "amount": 25.50,
      "date": "2022-01-12",
      "place": "Gas Station"
    },
    {
      "transaction_id": "3",
      "amount": 100.75,
      "date": "2022-01-15",
      "place": "Restaurant"
    }
  ];

  res.json(transactions);
});

app.listen(3000, () => {
  console.log('Server is running on port 3000');
});


### oefening

1. Creëer een prompt  die een databank schema genereert zodat het bovenstaande json formaat kan opgeslaan worden in een SQLite databank.
2. Creëer vervolgens een prompt die als datalayer van een NodeJs die data ophaalt.
3. En laat de LLM als laatste de door mij gegenereerde JavaScript code aanpassen zodat deze datalayer gebruikt wordt. 

In [ ]:
prompt = f"""Creeer een SQLite databank schema van onderstaande json data ```{code}```"""

response = get_answer(prompt)
print (response)


Het bovenstaande codefragment is een Node.js HTTP-server die een JSON-array van transacties retourneert wanneer een GET-verzoek naar de URL "/transactions" wordt gemaakt.

Om een SQLite-databaseschema te maken op basis van de JSON-gegevens, moeten we de structuur van de JSON begrijpen. Het lijkt erop dat elke transactie objecteigenschappen heeft zoals "transaction_id", "amount", "date" en "place". We kunnen deze eigenschappen gebruiken om de tabellen en kolommen van het SQLite-schema te maken.

Hier is een voorbeeld van een SQLite-databaseschema op basis van de JSON-gegevens:

```sql
CREATE TABLE transactions (
  transaction_id TEXT PRIMARY KEY,
  amount INTEGER,
  date TEXT,
  place TEXT
);
```

Het bovenstaande SQL-script maakt een tabel genaamd "transactions" met vier kolommen, namelijk "transaction_id", "amount", "date" en "place". De "transaction_id" kolom wordt gedefinieerd als een teksttype en ingesteld als de primaire sleutel. De "amount", "date" en "place" kolommen zijn ook ge

In [ ]:
prompt = f"""
Creëer een datalayer met NodeJs die data ophaalt uit een databank gecreeerd met het volgende schema ```CREATE TABLE transactions (
  transaction_id TEXT PRIMARY KEY,
  amount INTEGER,
  date TEXT,
  place TEXT
);```. Verander de code tussen triple backticks ```{code}```
"""

response = get_answer(prompt)
print (response)

Om een datalaag te maken in Node.js die gegevens ophaalt uit een database, moet je een databasebibliotheek installeren, zoals `pg` voor PostgreSQL-database.

Om te beginnen, installeer de `pg`-bibliotheek met npm:

```
npm install pg
```

Vervolgens kun je de volgende code gebruiken om de datalaag te maken:

```javascript
const http = require('http');
const { Client } = require('pg');

// Maak een nieuwe databaseclient
const client = new Client({
  user: 'database_user',
  host: 'localhost',
  database: 'database_name',
  password: 'database_password',
  port: 5432,
});

// Maak verbinding met de database
client.connect();

const server = http.createServer((req, res) => {
  if (req.url === '/transactions' && req.method === 'GET') {
    res.setHeader('Content-Type', 'application/json');

    // Query om transacties op te halen
    const query = 'SELECT * FROM transactions';

    // Voer de query uit
    client.query(query, (err, result) => {
      if (err) {
        console.error('Fout 

#### Role
Wanneer je de openai api gebruikt, dien je in de 'message' zowel een rol als een context mee te geven. De mogelijke rollen zijn: system, user of assistant.

De 'User' is de gebruiker. 'Assistant' is de chatbot (of ai assistant). Door beide rollen in een 'message' te gebruiken, kan je delen van eerdere conversaties terug als context meesturen bij een volgende vraag.

De context van de 'system' rol legt vast hoe het systeem zich dient te gedragen.

De voordelen van het specificeren van een rol zijn:

grotere context: Chatbots hebben een beperkte context, naarmate een conversatie vordert, vergeten ze wat ze eerder zeiden. Door het antwoord van de 'assistent' op te nemen in de volgende context, wordt dit minder snel vergeten.
standvastigheid: de rol die je vastlegt blijft makkelijker behouden, de chat messages met de gebruiker nadien gaan minder snel 'afleiden'.
Een (theoretisch) voorbeeld:



In [ ]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)

De huidige versie 'ChatGPT-4o' en zelfs de iets lichtere versie 'ChatGPT-4o-mini', zijn ondertussen wonderbaarlijk goed in het beantwoorden van opeenvolgende vragen. 

Hieronder volgen een aantal tips om goede prompts te schrijven, alsook enkele voorbeelden.

### principe 1 : prompts iteratief verbeteren

Als je prompts ontwerpt om dingen (programmatorisch) te bereiken met LLM's, zal je zelden van de eerste keer het resultaat bekomen dat je voor ogen had.
Het loont om de prompts iteratief te verbeteren, preciezer te maken in wat je wenst door de instructies te verscherpen, tot je het gewenste resultaat, in het gewenste formaat bereikt.


#### te lange output

In [ ]:
prompt = f"""
You're tasked to help the university college \
of ghent write a marketing piece for their \
website to promote the new course 'Trends in AI'

Describe the contents of the course and why it's \
really interesting for students to take this course
"""

res = get_answer(prompt)
print(res)

Title: Unlock the Future with 'Trends in AI' - Exploring the Revolutionary World of Artificial Intelligence

Introduction:
Welcome to the University College of Ghent, where innovation meets intellectual brilliance. Immerse yourself in the captivating realm of artificial intelligence (AI) through our cutting-edge course, 'Trends in AI.' Designed to equip students with foundational knowledge and practical skills, this course offers an unparalleled opportunity to explore and embrace the transformative power of AI.

Course Overview:
'Trends in AI' is a comprehensive exploration of the rapidly evolving field of artificial intelligence. From its fundamental principles to advanced applications, this course delves into the exciting world where human ingenuity intersects with machine intelligence. Combining theoretical knowledge with hands-on experiences, students develop the necessary skillset to leverage AI across various domains.

Key Highlights:
1. Introduction to AI: Gain a deep understand

In [ ]:
prompt = f"""
You're tasked to help the university college \
of ghent write a marketing piece for their \
website to promote the new course 'Trends in AI'

Describe the contents of the course and why it's \
really interesting for students to take this course

Use at most 25 words.
"""

res = get_answer(prompt)
print(res)

Explore cutting-edge applications of artificial intelligence and learn to analyze emerging trends. Gain a competitive edge in the digital era.


#### verkeerd doelpubliek

In [ ]:
prompt = f"""
You're tasked to help the university college \
of ghent write a marketing piece for their \
website to promote the new course 'Trends in AI'

Describe the contents of the course and why it's \
really interesting for students to take this course

Use at most 25 words, the description is intended for future students
"""

res = get_answer(prompt)
print(res)

"Discover the latest advancements in Artificial Intelligence in our new course. Gain insights, practical skills, and be at the forefront of innovation."


#### verkeerd formaat

In [ ]:
prompt = f"""
You're tasked to help the university college \
of ghent write a marketing piece for their \
website to promote the new course 'Trends in AI'

Describe the contents of the course and why it's \
really interesting for students to take this course

Use at most 25 words, the description is intended for future students, end with some relevant hashtags
"""

res = get_answer(prompt)
print(res)

Discover the cutting-edge world of artificial intelligence at University College of Ghent! Explore emerging trends and unlock limitless career opportunities. #AI #future #education


### principe 2: samenvattingen

LLM's zijn vrij goed in het samenvatten van grote(re) stukken tekst, maar je kan mits wat bijsturen de output vaak sterk verbeteren.

In [ ]:
text = f"""
wat

Prompt engineering is het sleutelen aan de vraag (de prompt) \
die je aan LLM's stelt zodat je een beter antwoord krijgt.

Er bleek al snel dat bij ChatGPT e.d. de bruikbaarheid en \
betrouwbaarheid van het antwoord sterk kan verbeterd worden \
door de vraag op een bepaalde manier te stellen. Er onstond \
zo een nieuwe discipline: prompt engineering.

De interesse hierin is zeer nieuw, en naarmate de modellen \
matuurder worden zal hier ook meer en meer consensus ontstaan \
van wat "best practices" zijn, maar we kunnen toch al een \
aantal principes opstellen over hoe je betere resultaten \
krijgt uit deze modellen, afhankelijk van wat je probeert \
te bereiken. De kans dat dit hoofdstuk volgend jaar (of zelfs \
tegen het einde van het semester) hetzelfde is, is vrij gering, \
dit onderzoeksdomein wijzigt zeer snel.

setup

We gaan met de OpenAI API connecteren, hiervoor heb je een OpenAI \
API key nodig, en een account bij OpenAI. De eerste €18 zijn gratis, \
maar je bent ook sowieso in tijd beperkt. (of je de €18 nu opgebruikt \
of niet, na drie maanden is de API niet langer gratis)

Om dit een beetje te duiden, je betaalt bij ChatGPT per 'token'. ChatGPT \
(en andere LLM's) genereren hun teksten in stukjes die we tokens noemen. \
Het hangt van de taal af, maar bij Engels en Nederlands komen tokens min \
of meer overeen met het aantal lettergrepen. Je betaalt zowel voor tokens \
in de vraag als in het antwoord aan een €0.002 per 1000 tokens. Je €18 is \
dus goed voor 9.000.000 tokens. Om wat in de les (en thuis) manueel mee te \
prutsen zal dit ruimschoots voldoende zijn, maar als je dit 'echt' gebruikt \
in geautomatiseerde processen best je verbruik goed in het oog houden.

Een alternatief is lokaal GPT4All draaien, die dezelfde API nabootst. De \
antwoorden zijn minder goed, maar het is wel altijd volledig gratis natuurlijk, \
dus zeker om alles op te zetten en te testen een aantrekkelijke optie. Ergens in \
juni werkte dit goed, begin september na een update kreeg ik het niet meer aan de \
praat op mijn macbook, hopelijk weer wel tegen dat jullie deze les krijgen. Maar \
los van de API draaien zijn er nog Python bindings, die wel werken, zie verder.

"""

In [ ]:
prompt = f"""
Summarize the text below, delimited by triple backticks, use at most 30 words

```{text}```
"""

res = get_answer(prompt)
print(res)

Prompt engineering involves manipulating the question (prompt) asked to an LLM to obtain better answers. It is a new discipline with evolving best practices. Using the OpenAI API incurs costs per token, but there are alternatives like running GPT4All locally.


In [ ]:
prompt = f"""
Summarize the text below, delimited by triple backticks, use at most 30 words, focus on the cost to the student.

```{text}```
"""

res = get_answer(prompt)
print(res)

Prompt engineering involves optimizing the question prompt to obtain better results. The cost to the student using ChatGPT is based on the number of tokens used, with the first €18 free but limited by time. An alternative is running GPT4All locally for free but with lower-quality results.


In [ ]:
prompt = f"""
Summarize the text below, delimited by triple backticks, use at most 30 words, focus on the novelty of the field.

```{text}```
"""

res = get_answer(prompt)
print(res)

The text discusses prompt engineering, a new discipline that focuses on improving the usefulness and reliability of answers generated by language models like ChatGPT through modifying the prompts. It also mentions the novelty and rapid evolution of the field. Additionally, it provides information on using the OpenAI API and the cost associated with using tokens. An alternative option is using a locally-run GPT4All, which is free but has slightly less accurate answers.


In [ ]:
prompt = f"""
Summarize the text below, delimited by triple backticks, use at most 30 words, focus on the cost, use bullet points to make the cost clear.

```{text}```
"""

res = get_answer(prompt)
print(res)

- Prompt engineering is the process of improving the usefulness and reliability of answers from language models.
- The cost of using the OpenAI API for ChatGPT is based on the number of tokens used in both the question and answer.
- The first €18 is free, but after three months, the API is no longer free.
- The cost is €0.002 per 1000 tokens.
- Using GPT4All locally is a free alternative with less accurate answers.
- Python bindings are also available for use.


### oefening

1. Creëer een prompt die de uitleg over LLM's van vorige week samenvat. (Zie 'text' hieronder).
2. Vervolgens willen we suggesties over hoe de uitleg kan verbeterd worden, en later ook uitgediept.
3. Daarnaast een prompt die alle schrijffouten eruit haalt, we willen enkel de tekst zonder schrijffouten, geen blabla er rond.


In [6]:
text = f"""
## model inference vs model training
 We spreken over:
- "model inference", d.i. het gebruik van het model => draait op een computer
- "model training", d.i. het leren van de parameters => dit is computationeel intensief.

De machine learning-modellen die generatieve AI ondersteunen, hebben deze vaardigheden geleerd door statistische patronen te vinden in enorme datasets met content die oorspronkelijk door mensen is gegenereerd. Grote taalmodellen zijn getraind op triljoenen woorden gedurende vele weken en maanden, en met veel rekenkracht. Deze foundation modellen, zoals we ze noemen, met miljarden parameters, vertonen eigenschappen die verder gaan dan alleen taal. Onderzoekers ontdekken hun vermogen om complexe taken te ontleden en problemen op te lossen.

Hier is een verzameling foundation modellen, soms ook basis modellen genoemd, en hun relatieve grootte in termen van parameters. Je zult later (eerder in het vak Deep Learning) in meer detail ingaan op deze parameters, maar je kan ze voorstellen als het geheugen van het model. Hoe meer parameters een model heeft, hoe meer geheugen, en zoals blijkt, hoe geavanceerder de taken die het kan uitvoeren. 
Stel dat we de LLMs voorstellen a.d.h.v. cirkels. Hier is een idee van het aantal parameters waarmee verschillende modellen getraind zijn:
- GPT3: 175 miljard
- BLOOM: 176 miljard
- LLama2: 70 miljard

![LLMs voorstelling](img/LLMs_voorstelling.png)

### voorbeeld
Hieronder is een voorbeeld om een gevoel te krijgen over wat het betekent in termen van resources om een model te trainen, dan wel een getraind model te gebruiken.

Om een 70b-large language model te draaien, heb je in principe 2 files nodig: 
- Een 140GB file die de model parameters bevat,
- Een file met +-500 lijnen code (for example C-code, or python code)

Nota: 70b staat voor "70 billion" of 70 miljard parameters. Voor elke parameter heb je 2 bytes nodig - er wordt gebruik gemaakt van "float16" data - dus de file wordt 140 GB groot.\
De code file dient om het language model te draaien, gebruik makend van de parameters. Dus, om gebruik te maken vaneen model, heb je in principe geen internet connectie nodig. Het draait op een (krachtige) computer.\
Je beschouwt de code file, compileert het, er wordt een binary file gemaakt die je naar de parameters kan laten verwijzen en je kan met dit "language model" praten.
Het "language model" gaat tekst genereren, het zal je instructies opvolgen.

![llama2_70b_2](img/llama2_70b_2.png)

Er is dus niet heel veel nodig om een model te gebruiken. De rekenkundige complexiteit komt erbij wanneer je die parameters wil bekomen, dus wanneer je het model traint. \
Wanneer je de parameters wilt leren, neem je "een deel van het internet" (om het concreet te maken, neem ongeveer 10 TB aan tekst). Om zo een hoeveelheid aan data te verwerken en de parameters van het neuraal netwerk te
leren, heb je ongeveer 6000 GPU's nodig, gedurende 12 dagen. De kost hiervoor wordt geschat op 2 miljoen dollar.

## Het model trainen
Twee belangrijke fasen in het trainen van een LLM zijn:
- het pre-trainen,
- het finetunen.

### LLM: trainingsfase 1 - het pre-trainen
Erg gesimplificeerd voorgesteld is het objectief van een LLM om het volgende woord in een zin/tekst te voorspellen.

Neem bijvoorbeeld de trainingszin “De lector sprak zijn _ aan”. Als in de oorspronkelijke tekst stond ‘student’, zal het model zijn gewichten zien versterken als hij student voorspelt.

Maar natuurlijk, de eerste L van LLM’s, de “Large”, maakt dat we uit zoveel data trainen dat voor vele zinnen er meerdere mogelijkheden zijn. Er is niet langer één juist antwoord, in het voorbeeld zou naast “student” ook “collega”, “opleidingshoofd”, “vriendin”, … een mogelijkheid kunnen zijn.
Dus bij het leren van de gewichten zullen we (conceptueel) een kans toewijzen aan elk van deze mogelijkheden, en dan zou ons model in een ideaal geval bijvoorbeeld 40% van de tijd student voorspellen, 20% van de tijd collega, 5% van de tijd vriendin enz.

Om een LLM dan langere teksten te laten genereren wordt de output na het genereren van een woord gebruikt als input voor de volgende voorspelling.

“De”  
“De lector”  
“De lector sprak”  

Het eindresultaat is een LLM die verbazend vlotte teksten kan genereren.
Dit pre-trained LLM noemen we een foundation model.

Hieronder wordt de pre-training fase schematisch voorgesteld:
![](/img/llm_pretraining_at_a_high_level_2.png)

Er wordt heel veel ongestructureerde data verzameld (teksten), bv. van internet. De kwaliteit van deze data laat echter te wensen over. Er blijkt dat slechts 1 tot 3% van de origineel verzamelde data bruikbaar is om van te leren (dus om het model op te trainen). Toch blijft dit nog een enorme hoeveelheid data. Met deze data wordt de pre-training gedaan, en er wordt een foundation model gecreëerd. Dit foundation model beschikt dan over een heel pak kennis, in de vorm van een 'vocabulair'. Voor deze pre-trainingsfase is heel veel rekenkracht (GPU) en tijd nodig.

In de pretrainingsfase spreken worden technieken als unsupervised learning en self-supervised learning gebruikt. Er komt geen menselijke feedback aan te pas.

### LLM: trainingsfase 2 - het finetunen
Maar, de teksten die het model genereert, zijn zeer afhankelijk van het soort tekst dat de trainingsdata vormde. Na het trainen van een foundation model, 'weet' dit model al veel, maar het is nog niet goed in het uitvoeren van specifieke taken, zoals bv. het samenvatten van tekst of het antwoorden op vragen. 
Als er veel vooroordelen in de oorspronkelijke tekst zaten, zullen die met een grote kans als ‘waarschijnlijk vervolg’ gekozen worden, en zal het large language model zonder veel moeite de meeste grove uitspraken maken.

Dus, er is een extra stap nodig: het model moet gefinetuned worden. Het model wordt opnieuw getraind met extra informatie. Men maakt datasets waarin het gewenste gedrag gereflecteerd wordt, en gebruikt die om het model verder op te trainen. HIerbij wordt ook gebruik gemaakt van menselijke feedback.

Deze extra informatie die gebruikt wordt voor finetuning is niet zo heel groot in volume (bv. 100000 vragen en bijhorende antwoorden), maar wel zeer kwalitatief voor de taak waarvoor men de LLM uiteindelijk wil gebruiken.
"""

### principe 3: intentie en gevoel

Soms willen we (snel) weten of een stuk tekst (bvb. een review) positief of negatief is. Ik kan me best voorstellen dat social media managers snel willen kunnen inspelen op zeer negatieve comments als er iets begint te 'leven' in de commentaren van een facebook post.

LLM's zijn hier ook vrij degelijk in, als voorbeeld heb ik een aantal reviews van een computermuis genomen van coolblue.

In [ ]:
review = f"""
Ideal mouse for photo editing for me and daily use with macOS. \
Format takes a bit of getting used to and it's a pity that they \
don't include the USB connector BOLT with the macOS version. \
Bluetooth connection is excellent. Use this mouse in combination \
with a Mac Studio. 
"""

In [ ]:
prompt = f"""
What is the sentiment of the following product review delimited with triple backticks

review: ```{review}```
"""
res = get_answer(prompt)
print(res)

The sentiment of the product review is positive.


In [ ]:
prompt = f"""
What is the sentiment of the following product review delimited with triple backticks

Give your answer with a single word, either "positive" of "negative"

review: ```{review}```
"""
res = get_answer(prompt)
print(res)

positive


In [ ]:
prompt = f"""
Identify a list of emotions that the writer of the review delimited with triple backticks is expressing.

Give no more than three emotions, as lower-case words separated by commas.

review: ```{review}```
"""
res = get_answer(prompt)
print(res)

satisfied, disappointed, content


#### oefening

Ga op zoek naar een lijst van reviews (vijf a tiental), maak een prompt die voor elke review een antwoord genereert als JSON object, met enerzijds een SENTIMENT key die 'positive' of 'negative' als antwoord geeft, en daarnaast EMOTIONS die een array bevat met de top drie emoties van de review.
Geef daarnaast ook een samenvatting van alle reviews 'mostly positive / mostly negative' al naargelang wat meest van toepassing is. (gegenereerd door een prompt uiteraard)

In [ ]:
prompt = f"""
Geef voor volgende reviews een JSON object met een SENTIMENT key (positief of negatief afhankelijk van de review) en EMOTIONS key dat een array bevat met 3 emoties die bij de review past. Geef hiernaast een samenvatting van alle reviews als: "mostly positive / mostly negative" afhankelijk van de reviews. 

review 1: 4/5 stars Ondanks dat de prestaties ERG goed zijn voor creatieve programmas zoals, photoshop/illustrator/aftereffects, is deze laptop NIET geschikt voor gaming. Laat je daarom ook NIET gek maken door de specificaties en gebruik hoogstens alleen LICHTE games op deze laptop. Op het oog is het design/ontwerp echt heel mooi, heel compact en ben ik er echt tevreden mee. Daarentegen is het scherm (ook door eerdere reviews al gezecht) HEEL kwetsbaar voor vingerafdrukken en stof. Haal het stof er gewoon af met een stoffen doekje en niet met je handen. Ook maakt het materiaal het niet best voor het scherm gedeelte. Het krijgt zeer makkelijk beschadigingen, met name bij het gedeelte waar het scherm rond het toetsenbord draait. Wanneer de laptop dicht is zit er een klein openingetje tussen het scherm en het toetsenbord (wat gewoon zo hoort), maar als je deze erg word belast, onder andere door in een te volle tas te doen, word het scherm op het toetsen word gedrukt en blijft deze zo staan en komen er Allerlei beschadigingen rond het draaigedeelte (foto). En leg er ZEKER niks zwaars op! Ook tasten deze beschadigingen het scherm zelf iets aan. Conclusie, Ik zou deze zekerrrrr aanbevelen voor creatieve programmas en uiterst lichte games, maar alsnog, ga er echt voorzichtig mee om, het beschadigd makkelijker dan je denkt. 
review 2: 5/5 stars Vorige week deze Vivobook van Asus gekocht. Naast dat de laptop een erg mooi design heeft, is hij ook super snel dankzij de i7 processor en 16gb werkgeheugen en de videokaart. Ik bewerk graag foto's en video's en zelfs dat verloopt soepeltjes. Zeker een aanrader wanneer je grafische taken wilt uitvoeren. 
review 3: 3/5 stars Valt dat ff tegen! Mijn 10 jaar oude computer, (wel met 2 ssd schijven) is binnen 20 seconden volledig opgestart. Daar kan deze niet aan tippen!! Ook niet na het uitschakelen van programma's die dat nog meer vertragen. Eenmaal opgestart is hij wel lekker snel maar ook het uit slaapstand komen is ook weer erg traag. 
review 4: 5/5 stars Het is een zeer goede laptop met een goede processor. Je kan er wat games op spelen die niet te zwaar zijn. De SSD en de i7 processor combinatie maakt de laptop erg snel. Vingerafdruk scanner is snel en is mooi in het TouchPad verwerkt. Toetsenbord geeft licht. Laptop start binnen 2-5 seconden op. 
review 5: 4/5 stars Na lang twijfelen heb ik er toch voor gekozen om deze laptop te kiezen en ik ben er zeer blij mee. De laptop heeft een mooi ontwerp en goede specificaties. De game kan ook bepaalde lichte games runnen zoals Minecraft. Voor multitasken en media bewerking is deze laptop ook zeer fijn. De enige echte minpunten die ik kan noemen zijn de hele slechte speakers (geloof me, het is slechter dan je denkt) en de helderheid van het scherm (die valt wel mee). Voor mij is dit verder niet echt een probleem en voor de audio gebruik ik toch altijd een headset. Al met al is het een mooi product."""

res = get_answer(prompt)
print(res)

{
  "review1": {
    "SENTIMENT": "negatief",
    "EMOTIONS": ["frustratie", "tevredenheid", "voorzichtigheid"]
  },
  "review2": {
    "SENTIMENT": "positief",
    "EMOTIONS": ["tevredenheid", "enthousiasme", "aanbeveling"]
  },
  "review3": {
    "SENTIMENT": "negatief",
    "EMOTIONS": ["teleurstelling", "frustratie", "traagheid"]
  },
  "review4": {
    "SENTIMENT": "positief",
    "EMOTIONS": ["tevredenheid", "snelheid", "gemak"]
  },
  "review5": {
    "SENTIMENT": "positief",
    "EMOTIONS": ["blijdschap", "tevredenheid", "gemak"]
  }
}

Samenvatting: mostly positive


#### principe 4: spelling en stijl

LLM's blijken uitstekende spell checkers te zijn, maar zoals altijd, niet blindelings te vertrouwen. Daarnaast blijken LLM's ook goed in een bepaalde stijl te kunnen schrijven, wat erg handig kan zijn als je een tekst formeler wilt laten klinken (of net niet).

In [ ]:
text = f"""
Dit onderzoek omvat een casus van Randstad/Tempo team waar er met een dataset \
die netwerkverkeer van alle kantoren bevat, een grondige en correcte analyse \
gaan gemaakd worden. Op basis van die analyse kunnen er te weten gekomen worden \
wat er in een kantoor gebeurt en kunnen er zo conclusies getrokken worden over de werkdruk. \
Het is voor het bedrijf om belangrijk te weten te komen of er al dan niet teveel werknemers \
op een bepaald kantoor werken en zo onnodig bronnen besteed worden waar het niet nodig is en \
dan die bronnen kunnen gebruiken waar wel het nodig is. De aanleiding kwam vanuit de CIO van \
het bedrijf zelf. Hij wou dit omdat men zoveel kantooren over heel het land heeft en dat dit \
niet manueel te controleren valt. Hij wil dit kunnen controleren met data afkomstig van het \
netwerkverkeer om zo een beeld te krijgen van wat er gepresteerd wordt op een kantoor. \
De data die we van het bedrijf krijgen, kunnen we aan de hand van filtering bruikbaar maken voor analyse. \
Uiteindelijk is het de bedoeling om een duidelijk overzicht krijgen met onder andere een proof of concept.
"""


In [ ]:
prompt = f"""
Identificeer alle schrijffouten in de volgende tekst, afgebakend met triple backticks. \
Lijst ze op per categorie (spelfouten / grammaticale fouten / fouten tegen vervoegingen) en geef ook de gecorrigeerde tekst.

tekst: ```{text}```
"""

res = get_answer(prompt)
print(res)

Spelfouten:
- aangepaste (gemaakd)
- voor het bedrijf om belangrijk (voor het bedrijf is het belangrijk)
- teveel werknemers (te veel werknemers)
- het niet nodig is en dan die (en die)
- die bronnen kunnen gebruiken waar wel het nodig is (die bronnen kunnen gebruiken waar het wel nodig is)
- zoveel kantooren (zoveel kantoren)

Grammaticale fouten:
- waar er met een dataset die netwerkverkeer (waarbij een dataset wordt gebruikt die netwerkverkeer)
- conclusies getrokken worden over de werkdruk (conclusies worden getrokken over de werkdruk)
- te weten gekomen worden wat er in een kantoor gebeurt (te weten gekomen kan worden wat er in een kantoor gebeurt)
- besteed worden waar het niet nodig is en dan die bronnen kunnen gebruiken waar wel het nodig is (besteed worden waar het niet nodig is en die bronnen kunnen gebruiken waar het wel nodig is)

Fouten tegen vervoegingen:
- gaan gemaakd worden (zullen gemaakt worden)
- te weten gekomen worden (te weten gekomen worden)
- conclusies kunnen 

In [ ]:

prompt = f"""
Je bent een bachelorstudent aan de hogeschool, herschrijf de hierna volgende introductie \
van een onderzoeksvoorstel, afgebakend door triple backticks. \
Herschrijf enkel de introductie in een meer formele onderzoeksstijl, zonder schrijffouten, \
en doe dit zin per zin zodat de oorspronkelijke structuur behouden blijft.

voorstel: ```{text}```
"""

res = get_answer(prompt, False)
print(res)

Het voorliggende onderzoek betreft een casestudie van Randstad/Tempo-team, waarbij een grondige en nauwkeurige analyse wordt uitgevoerd van een dataset die het netwerkverkeer van alle kantoren bevat. Op basis van deze analyse is het mogelijk om inzicht te verkrijgen in de activiteiten die plaatsvinden in een kantoor en om conclusies te trekken met betrekking tot de werkdruk. Het is van belang voor het bedrijf om vast te stellen of er mogelijk te veel werknemers werkzaam zijn op bepaalde kantoren, zodat onnodige middelen kunnen worden vermeden en deze middelen efficiënter kunnen worden ingezet op de juiste locaties. De impuls voor dit onderzoek is afkomstig van de CIO van het bedrijf zelf, die van mening is dat handmatige controle van alle kantoren verspreid over het hele land niet haalbaar is. Hij wenst de mogelijkheid te hebben om aan de hand van gegevens afkomstig van het netwerkverkeer een beeld te verkrijgen van de prestaties op een kantoor. Door middel van filtering kunnen we de o

In [ ]:

prompt = f"""
Je bent een bachelorstudent aan de hogeschool, herschrijf de hierna volgende introductie \
van een onderzoeksvoorstel, afgebakend door triple backticks. \
Herschrijf enkel de introductie in een meer formele onderzoeksstijl, zonder schrijffouten.

voorstel: ```{text}```
"""

res = get_answer(prompt, False)
print(res)

Voorstel:

```
Dit onderzoek betreft een casestudy van Randstad/Tempo Team, waarbij een grondige en correcte analyse wordt uitgevoerd op een dataset die het netwerkverkeer van alle kantoren omvat. Op basis van deze analyse kan inzicht worden verkregen in de activiteiten op een kantoor, waardoor conclusies kunnen worden getrokken met betrekking tot de werkdruk. Het is voor het bedrijf van belang te weten te komen of er mogelijk te veel werknemers werkzaam zijn op bepaalde kantoren, waardoor onnodige middelen worden besteed op plaatsen waar dit niet noodzakelijk is. Deze middelen kunnen dan beter ingezet worden op de plekken waar dit wel noodzakelijk is. De aanleiding voor dit onderzoek kwam vanuit de Chief Information Officer (CIO) van het bedrijf zelf. Hij heeft behoefte aan een methode om dit te controleren omdat er verspreid over het hele land zoveel kantoren zijn, wat handmatige controle onmogelijk maakt. Hij wil in staat zijn om met behulp van data afkomstig van het netwerkverkeer 

Qua stijl kan je dus makkelijk formeel taalgebruik vragen, maar je kan hier ver in gaan, heel ver

In [ ]:
prompt = f"""
Je bent een bachelorstudent aan de hogeschool, herschrijf de hierna volgende introductie \
van een onderzoeksvoorstel, afgebakend door triple backticks. \
Herschrijf enkel de introductie als monty python sketch.

voorstel: ```{text}```
"""

res = get_answer(prompt, False)
print(res)

"Scene: The offices of Randstad/Tempo Team. A group of silly researchers are gathered around a computer, huddled over a dataset of network traffic.

Researcher 1: (excitedly) Good morning, everyone! Today, we embark on a tremendous adventure in the land of data analysis!

Researcher 2: (curiously) What are we analyzing this time?

Researcher 1: (grinning) Oh, you won't believe it! We have been entrusted with the task of unraveling the mysteries of the network traffic in all the offices of Randstad/Tempo Team!

Researcher 3: (sarcastically) Oh joy, traffic analysis. How riveting!

Researcher 1: (ignoring the sarcasm) But wait, my friends, this is not just any data analysis. With our sharp minds and impeccable algorithms, we shall uncover the secrets of what truly happens in these offices. And more importantly, we shall determine the workload!

Researcher 4: (intrigued) The workload, you say? How on earth can we do that?

Researcher 1: Ah, my dear colleague, fear not! Our trusty dataset 

In [ ]:
prompt = f"""
Je bent een bachelorstudent aan de hogeschool, herschrijf de hierna volgende introductie \
van een onderzoeksvoorstel, afgebakend door triple backticks. \
Herschrijf enkel de introductie in de smurfentaal.

voorstel: ```{text}```
"""

res = get_answer(prompt, False)
print(res)

Voorstel: ```
Dit onderzoek gaat over een smurfse casus van Randstad/Tempo team waar er smurfgehoopt wordt om met een smurftastische dataset die smurfig netwerkverkeer van alle smurfkantoren bevat, een grondige en smurfcorrecte analyse te smurfen. Op basis van die smurflistige analyse kunnen er smurfwijze conclusies worden getrokken over de smurfwerkdruk en wat er allemaal smurfgebeurt in een kantoor. Het is voor het smurfbedrijf heel belangrijk om te weten te smurfen of er al dan niet teveel smurfen op een bepaald kantoor werken, zodat ze geen smurfbelangrijke smurfmiddelen versmurfen waar het niet nodig is, maar juist smurfbestemmen waar het wel nodig is. De smurfaanleiding voor dit onderzoek komt vanuit de smurfCIO van het smurfbedrijf zelf. Hij wilde dit omdat er zoveel smurfkantoren verspreid zijn over het hele smurfland, en dat het smurfcontroleren hiervan met de hand onsmurfelijk is. Hij wil deze smurfcontrole kunnen uitvoeren met data die afkomstig is van het smurfnetwerkverkee

In [ ]:
prompt = f"""
Je bent een bachelorstudent aan de hogeschool, herschrijf de hierna volgende introductie \
van een onderzoeksvoorstel, afgebakend door triple backticks. \
Herschrijf enkel de introductie als limerick.

voorstel: ```{text}```
"""

res = get_answer(prompt, False)
print(res)

Er was eens een onderzoek in Randstad,
Met data van elke kantoortje gegrond.
We willen weten,
Hoe werkdruk te meten.
En zo onnodige kosten besparen, blond.


Afhankelijk van wie jullie promoter is, misschien een ideetje voor de bachelorproef?

#### principe 5: uitwerken

LLM's kunnen niet enkel samenvatten, ook uitweiden is een optie.

In [ ]:
gevoel = "negatief"
inhoud = "tweede les trends in ai, weinig nieuws geleerd, veel te langdradig, te weinig oefeningen"

In [ ]:
prompt = f"""
Stel een beleefde email op, zowel het gevoel als de korte inhoud worden hierna tussen triple backticks gegeven.


gevoel: ```{gevoel}```
inhoud: ```{inhoud}```
"""

mail = get_answer(prompt, False)
print(mail)

Beste docent,

Ik hoop dat deze email u goed bereikt. Ik wil graag mijn feedback delen over de tweede les trends in AI. Ten eerste wil ik benadrukken dat ik het belangrijk vind om constructieve kritiek te geven, want ik geloof dat dit kan bijdragen aan een betere leerervaring voor ons allemaal.

Helaas moet ik zeggen dat ik niet erg tevreden was met de tweede les. Ik had gehoopt veel nieuwe dingen te leren, maar helaas bleek dit niet het geval te zijn. Ik vond de les te langdradig en ik merkte dat ik hierdoor mijn concentratie verloor. Daarnaast vond ik dat er te weinig oefeningen werden gegeven, waardoor ik het gevoel had dat ik de stof niet goed kon toepassen.

Ik begrijp dat het onderwerp trends in AI uitgebreid is, maar ik denk dat het mogelijk is om de les meer interactief en boeiend te maken. Wellicht kunnen er meer praktijkvoorbeelden worden gegeven en kunnen we actief deelnemen aan discussies om de leerervaring te versterken.

Ik wil graag benadrukken dat ik de lessen oprecht w

En als docent lezen we de mail niet meer natuurlijk, maar doen we net het omgekeerde

In [ ]:
prompt = f"""
Vat de mail tussen triple backticks samen in een aantal bulletpoints, geef ook het algemene gevoel van de mail als 'positief', 'negatief' of 'neutraal'

mail: ```{mail}```
"""

res = get_answer(prompt, False)
print(res)

- Feedback over de tweede les trends in AI
- Belang van constructieve kritiek
- Ongenoegen over de les
- Les was te langdradig en concentratie werd verloren
- Gebrek aan oefeningen om de stof toe te passen
- Suggestie om de les interactiever en boeiender te maken met praktijkvoorbeelden en discussies
- Waardering voor de lessen en de tijd en moeite van de docent
- Constructieve feedback om de lessen te verbeteren
- Uitkijken naar de volgende les voor een meer boeiende en interactieve leerervaring

Algemeen gevoel van de mail: negatief


In dit geval toch al een upgrade van negatief naar neutraal, ChatGPT staat aan de zijde van de docent.

### oefening

Creëer een prompt die op een heel positieve manier suggesties geeft over hoe de verdere cursus 'Trends in AI' kan verbeterd worden.
(en als er iets goed uitkomt, mail gerust 😉)